# Problem Set 3 CS129.18

The following problem set will revolve around the Enron Emails dataset.
The dataset `data/enron-data/` directory has 6 files. The objective is to build a classifier for whether email is spam or not.


## Requirements


>Spam or Ham?

Using the lessons on Naive Bayes and TF-IDF, and the other resource notebooks, show the following in this Jupyter Notebook.

Write your answers down as Markdown cells or comments in the code.

**Using Enron 1 and 2**

1. How many Spam Emails are there? ( 1 pt )

2. Structure the email data from the 3 directories into 1 dataframe with columns: Status, Subject, Body ( 7 pts )

3. Build a Naive Bayes classifier to classify whether emails are spam or not. ( 3 pts )

4. What is the longest ham email? ( 1 pt )

5. What is the accuracy of your model?( 1 pt )

6. Include the Subject in the analysis of the emails, does the accuracy/performance of the model increase? (7 pts)



-----


Bonus : Answer questions 1-6 using Enron 1,2, and 3. (5 pts)

----

**Submit this file on Moodle on the submission link I will provide. This is due October 18 12nn.**

In [1]:
%matplotlib inline

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import os
import random

import dask.dataframe as dd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
rootdirs = [
    "data/enron-data/enron1",
    "data/enron-data/enron2",
    "data/enron-data/enron3",
#     "data/enron-data/enron4",
#     "data/enron-data/enron5",
#     "data/enron-data/enron6",
]

# Loop through all the directories, sub directories and files in the above folder, and print them.
for rootdir in rootdirs:
    for directories, subdirs, files in os.walk(rootdir):
        print(directories, subdirs, len(files))

data/enron-data/enron1 ['ham', 'spam'] 2
data/enron-data/enron1\ham [] 3672
data/enron-data/enron1\spam [] 1500
data/enron-data/enron2 ['ham', 'spam'] 1
data/enron-data/enron2\ham [] 4361
data/enron-data/enron2\spam [] 1496
data/enron-data/enron3 ['ham', 'spam'] 1
data/enron-data/enron3\ham [] 4012
data/enron-data/enron3\spam [] 1500


In [3]:
ham_list = []
spam_list = []

# Same as before, but this time, read the files, and append them to the ham and spam list
for rootdir in rootdirs:
    for directories, subdirs, files in os.walk(rootdir):
        if (os.path.split(directories)[1]  == 'ham'):
            for filename in files:      
                with open(os.path.join(directories, filename), encoding="latin-1") as f:
                    data = f.read()
                    ham_list.append(data)

        if (os.path.split(directories)[1]  == 'spam'):
            for filename in files:
                with open(os.path.join(directories, filename), encoding="latin-1") as f:
                    data = f.read()
                    spam_list.append(data)


print(len(ham_list))
print(len(spam_list))

12045
4496


In [4]:
# 1. Number of Spam Emails
print('There are {spam} spam emails'.format(spam=len(spam_list)))

There are 4496 spam emails


In [5]:
#2. Structure Data
data = {
    'status': [],
    'subject': [],
    'body': [],
    'raw': [],
    'original': [],
}

import re

def mike_clean(string):
    return re.sub(r"[^a-zA-Z0-9]", r" ", string.strip())

def mike_append(status, the_list):
    for string in the_list:
        # removes Subject: fw: or re:
        subject = mike_clean(re.sub(r"^Subject:(.{0,4}\:){0,1}", r"", string.split('\n', 1)[0]))
        body = mike_clean(''.join(string.split('\n', 1)[1:]))
        data['status'].append(status)
        data['subject'].append(subject)
        data['body'].append(body)
        data['raw'].append(subject + " " + body)
        data['original'].append(string)

mike_append("ham", ham_list)
mike_append("spam", spam_list)

df = pd.DataFrame(data=data)
df.head()

,status,subject,body,raw,original
0,ham,christmas tree farm pictures,,christmas tree farm pictures,Subject: christmas tree farm pictures\n
1,ham,vastar resources inc,gary production from the high island larger ...,vastar resources inc gary production fro...,"Subject: vastar resources , inc .\ngary , prod..."
2,ham,calpine daily gas nomination,calpine daily gas nomination 1 doc,calpine daily gas nomination calpine daily g...,Subject: calpine daily gas nomination\n- calpi...
3,ham,issue,fyi see note below already done stella ...,issue fyi see note below already done st...,Subject: re : issue\nfyi - see note below - al...
4,ham,meter 7268 nov allocation,fyi ...,meter 7268 nov allocation fyi ...,Subject: meter 7268 nov allocation\nfyi .\n- -...


In [6]:
len(df)

16541

In [7]:
train_x, test_x, train_y, test_y = train_test_split(data["body"], data["status"], test_size=0.33, random_state=2574)
print("split")
vectorizer = TfidfVectorizer()
tfidf_train_x = vectorizer.fit_transform(train_x)
print("vectorize")

index_value={i[1]:i[0] for i in vectorizer.vocabulary_.items()}
fully_indexed = []
for row in tfidf_train_x:
    fully_indexed.append({index_value[column]:value for (column,value) in zip(row.indices,row.data)})
print("indexed")

# 3. NB
classifier = GaussianNB()
classifier.fit(tfidf_train_x.toarray(), train_y)
print("classifier")

#5. Accuracy

tfidf_test_x = vectorizer.fit_transform(test_x)
print(tfidf_test_x.shape)
scores = cross_val_score(classifier, tfidf_test_x.toarray(), test_y, cv=5)
acc = scores.mean()
print("Accuracy: %0.2f percent" % (acc *100))

split
vectorize
indexed
classifier
(5459, 55075)
Accuracy: 93.53 percent


In [8]:
# 4. Longest Ham Email

ham_list.sort(key=len)
longest_email = ham_list[len(ham_list) - 1]

print('The longest ham email is {len} characters long.'.format(len=len(longest_email),email=longest_email))

The longest ham email is 228377 characters long.


In [9]:
# 6. With Subject

train_x, test_x, train_y, test_y = train_test_split(data["raw"], data["status"])
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_train_x = vectorizer.fit_transform(train_x)

index_value={i[1]:i[0] for i in vectorizer.vocabulary_.items()}
fully_indexed = []
for row in tfidf_train_x:
    fully_indexed.append({index_value[column]:value for (column,value) in zip(row.indices,row.data)})

classifier = GaussianNB()
classifier.fit(tfidf_train_x.toarray(), train_y)

tfidf_test_x = vectorizer.transform(test_x)
print(tfidf_test_x.shape)
scores = cross_val_score(classifier, tfidf_test_x.toarray(), test_y, cv=5)
acc = scores.mean()
print("Accuracy: %0.2f percent" % (acc *100))

# Answer: Parsing the raw body including the subject gives 
# the model more text to work with. While the improvement 
# may be little, there certainly is one. Adding the subject 
# improves its accuracy. 

(4136, 84581)
Accuracy: 93.33 percent
